# 1 Author

 Samarth Khare                                               


# 2 Problem formulation

A Machine learning pipeline needs to be build,using the MLEnd Hums and Whistles dataset, which takes Potter or Starwars audio and predict its song label.

# 3 Machine Learning pipeline

I have used 219 wav audio files of Potter and Starwars song to build pipeline and validate chosen model.Since raw audio signals are made up of thousands of dimensions, but due to limited no. of training data and computation power,pipeline transformed each audio signal into 4 dimension and 1 corresponding label by  extracting same features that we used in starter kit namely Power,Pitch mean,Pitch standard deviation,Fraction of voiced region.

# 4 Transformation stage

In transformation stage, I used four features namely Power,Pitch mean,Pitch standard deviation,Fraction of voiced region of every audio signal. which is used as predictor.

# 5 Modelling
While I tried to train different models like SVM, KNN, Random Forest classifier with multiple combination of predictors  like the one present in the starter kit or mfcc or combination of both but validation accuracy was looming in between 40-60% for all the classifier mentioned. finally I have chosen KNN since its validation accuracy was bit better than the rest. 

# 6 Methodology

I have divided the data into training and testing set , and using accuracy to asses model performance.

#Code

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, sys, re, pickle, glob
import urllib.request

import IPython.display as ipd
from tqdm import tqdm
import librosa

drive.mount('/content/drive')

Mounted at /content/drive


Below code shows us the count of audio files present in given folder

In [ ]:
files_path = '/content/drive/MyDrive/basic/MLEndHW/sample2/*.wav'
files = glob.glob(files_path)
print("Number of Audio files :" ,len(files))

Number of Audio files : 219


creating a list containing files name and song name

In [ ]:
MLENDHW_table = [] 

for file in files:
  file_name = file.split('/')[-1]
  song = file.split('/')[-1].split('_')[3].split('.')[0]
  MLENDHW_table.append([file_name, song])

MLENDHW_table

[['S24_whistle_2_Potter.wav', 'Potter'],
 ['S29_whistle_2_Potter.wav', 'Potter'],
 ['S34_whistle_2_Potter.wav', 'Potter'],
 ['S32_whistle_2_Potter.wav', 'Potter'],
 ['S40_whistle_2_Potter.wav', 'Potter'],
 ['S39_whistle_2_Potter.wav', 'Potter'],
 ['S43_whistle_2_Potter.wav', 'Potter'],
 ['S45_whistle_2_Potter.wav', 'Potter'],
 ['S44_whistle_2_Potter.wav', 'Potter'],
 ['S48_Whistle_2_Potter.wav', 'Potter'],
 ['S50_whistle_2_Potter.wav', 'Potter'],
 ['S52_whistle_2_Potter.wav', 'Potter'],
 ['S54_whistle_2_Potter.wav', 'Potter'],
 ['S56_whistle_2_Potter.wav', 'Potter'],
 ['S57_whistle_2_Potter.wav', 'Potter'],
 ['S64_whistle_2_Potter.wav', 'Potter'],
 ['S66_whistle_2_Potter.wav', 'Potter'],
 ['S67_whistle_2_Potter.wav', 'Potter'],
 ['S68_whistle_2_Potter.wav', 'Potter'],
 ['S72_whistle_2_Potter.wav', 'Potter'],
 ['S76_whistle_2_Potter.wav', 'Potter'],
 ['S75_whistle_2_Potter.wav', 'Potter'],
 ['S77_whistle_2_Potter.wav', 'Potter'],
 ['S80_whistle_2_Potter.wav', 'Potter'],
 ['S78_whistle_2

creating dataframe and making file name as its index

In [ ]:
MLENDHW_df = pd.DataFrame(MLENDHW_table,columns=['file_id','song']).set_index('file_id') 
MLENDHW_df

,song
file_id,
S24_whistle_2_Potter.wav,Potter
S29_whistle_2_Potter.wav,Potter
S34_whistle_2_Potter.wav,Potter
S32_whistle_2_Potter.wav,Potter
S40_whistle_2_Potter.wav,Potter
...,...
S106_hum_1_Potter.wav,Potter
S106_hum_2_Potter.wav,Potter
S106_hum_3_StarWars.wav,StarWars


converting values of song column to uppercase

In [ ]:
MLENDHW_df['song'] = MLENDHW_df['song'].str.upper()
MLENDHW_df

,song
file_id,
S24_whistle_2_Potter.wav,POTTER
S29_whistle_2_Potter.wav,POTTER
S34_whistle_2_Potter.wav,POTTER
S32_whistle_2_Potter.wav,POTTER
S40_whistle_2_Potter.wav,POTTER
...,...
S106_hum_1_Potter.wav,POTTER
S106_hum_2_Potter.wav,POTTER
S106_hum_3_StarWars.wav,STARWARS


Feature extraction - function below is used to get pitch of an audio signal.

In [ ]:
def getPitch(x,fs,winLen=0.02):
  #winLen = 0.02 
  p = winLen*fs
  frame_length = int(2**int(p-1).bit_length())
  hop_length = frame_length//2
  f0, voiced_flag, voiced_probs = librosa.pyin(y=x, fmin=80, fmax=450, sr=fs,
                                                 frame_length=frame_length,hop_length=hop_length)
  return f0,voiced_flag

Function to return predictor and label after looping through each sample.

In [ ]:
def getXy(files,labels_file, scale_audio=False, onlySingleDigit=False):
  X,y =[],[]
  for file in tqdm(files):
    fileID = file.split('/')[-1]        
    yi = labels_file.loc[fileID]['song']=='POTTER'
    fs = None # if None, fs would be 22050
    x, fs = librosa.load(file,sr=fs)
    if scale_audio: x = x/np.max(np.abs(x))
    f0, voiced_flag = getPitch(x,fs,winLen=0.02)
      
    power = np.sum(x**2)/len(x)
    pitch_mean = np.nanmean(f0) if np.mean(np.isnan(f0))<1 else 0
    pitch_std  = np.nanstd(f0) if np.mean(np.isnan(f0))<1 else 0
    voiced_fr = np.mean(voiced_flag)

    xi = [power,pitch_mean,pitch_std,voiced_fr]
    X.append(xi)
    y.append(yi)

  return np.array(X),np.array(y)

In [ ]:
X,y=getXy(files, labels_file=MLENDHW_df, scale_audio=True, onlySingleDigit=True)

100%|██████████| 219/219 [11:43<00:00,  3.21s/it]


Below dataframe contains the four features values and label for each sample.

# 7 Dataset

I splited the original dataset of 219 audio into training to train the model and test to validate its accuracy.

Below dataframe contains the 4 features values and label for each sample.

In [ ]:
df_features = pd.DataFrame(X,columns=['feature_1','feature_2','feature_3','feature_4'])
df_features['labels']=y
df_features

,feature_1,feature_2,feature_3,feature_4,labels
0,0.019528,401.377634,34.522270,0.544214,True
1,0.011814,396.268476,33.924044,0.542453,True
2,0.025150,413.732772,20.644287,0.551938,True
3,0.055641,410.810178,23.655076,0.759193,True
4,0.004164,393.561321,32.013405,0.651045,True
...,...,...,...,...,...
214,0.183165,237.172077,60.371162,0.913846,True
215,0.077045,218.721353,58.457676,0.911402,True
216,0.060811,219.751489,50.264046,0.874325,False
217,0.082059,223.939810,51.127465,0.830621,False


shape of predictor and label

In [ ]:
print('The shape of X is', X.shape) 
print('The shape of y is', y.shape)

The shape of X is (219, 4)
The shape of y is (219,)


In [ ]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#Xs = scaler.fit_transform(X)

count of each class

In [ ]:
print(' The number of Potter recordings is ', np.count_nonzero(y))
print(' The number of Starwars recordings is ', y.size - np.count_nonzero(y))

 The number of Potter recordings is  110
 The number of Starwars recordings is  109


Split the data into training and testing (70:30 ratio)

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((153, 4), (66, 4), (153,), (66,))

class GridSearchCV offers a convenient way to choose hyperparameters based on cross-validation. its constructor receives a classification algorithm object and a dictionary that maps hyperparameter names to lists of values that should be considered and parameter cv controls the number of folds K.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

parameters = {'n_neighbors': [3,4,5]}

knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, parameters, cv=5)
knn_cv.fit(X_train, y_train)



GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 4, 5]})

# 8 Results

GridsearchCV gives the best hyperparameter setting and training and testing accuracy. Here, Knn with parameter of n_neighbours = 4 is turning out to be best fit.

In [ ]:
print('Best hyperparameter setting: {0}.'.format(knn_cv.best_estimator_))
print('Training dataset accuracy of best hyperparameter setting: {0}.'.format(knn_cv.score(X_train, y_train)))
print('Test dataset accuracy of best hyperparameter setting: {0}.'.format(knn_cv.score(X_test, y_test)))

Best hyperparameter setting: KNeighborsClassifier(n_neighbors=4).
Training dataset accuracy of best hyperparameter setting: 0.7647058823529411.
Test dataset accuracy of best hyperparameter setting: 0.696969696969697.


# 9 Conclusions

Test accuracy of knn classifier (n_neighbours = 4) is 69%, model is not giving  high accuracy , also training accuracy is also not high, that may signify that model is underfitting , and  may require further exploration in terms of feature extraction and complex model selection.